# Projet maison n° 4

In [1]:
# imports
import requests
from bs4 import BeautifulSoup
import re

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [2]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    # je récupère l'url de la page concerné
    r = requests.get(url)
    # je récupère le text html de la page avec les balises
    soup = BeautifulSoup(r.text)
    # Le tableau avec les données qui nous intéressent se trouve dans la class pricing tabel
    # On a tout le contenu de la balise correpsondante
    table_rows = soup.select(".pricing-table")
    # On va poarcourir chaque ligne de notre texte
    for table_row in table_rows:
        # On va commencer à récuperer les infos demandés
        # Tout d'abord le nom de notre petit tableau
        key = table_row.find('h2').text
        # on cherche le prix donc on veut un dollard puis des chiffres
        # le prix se trouve dans un <span> 
        price = re.search('[$][0-9]+',table_row.find('span').text)[0]
        # on l'ajoute dans notre dico avec en clé le nom du tableau dans lequel ca se situe
        prices[key] = {'price':price}
        # le principe est similaire au prix, on ne s'embete pas à chercher un endoit
        # on parcourt entierement le texte et si on trouve des chiffres + Un B à la fin on récupère
        storage = re.search('[0-9]+.B',table_row.text)[0]
        # on stocke
        prices[key]['storage'] = storage
        # Meme principe que storage, sauf que double recherche, première pour trouver (X databeses)
        # une deuxième pour ne prendre que la valeur, [0] permet de récupérer la valeur et non le résultat de la fonction re
        databases = re.findall('[0-9]+',re.search('[0-9]+ database',table_row.text)[0])[0]
        # ajout de databases
        prices[key]['databases'] = int(databases)
    return prices

print(get_prices_from_url(URL_PAGE2))
print(get_prices_from_url(URL_PAGE3))

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1}, 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5}, 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}, 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}


**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [3]:
# partie B-1
url = "https://www.beerwulf.com/fr-fr/p/bieres/newcastle-brown-ale-fut-2l"

# partie B-1
def extract_beer_infos(url):
    # on récupère la page
    r = requests.get(url)
    
    # on recupère le contenue
    soup = BeautifulSoup(r.text)

    # récupérer le titre qui se trouve dans la class product-detail-info-title
    title = [x.find('h1') for x in soup.find_all(class_='product-detail-info-title')][0]
    # on doit enlever la première et derniere ligne de l'info en position [0]
    style = soup.find_all('dd', class_=['small-6 medium-9 columns'])[0].text.replace("\n", "")
    # on recup l'info en [1] mais un peu de transformation pour avoir le format désiré
    degre = float(soup.find_all('dd', class_=['small-6 medium-9 columns'])[1].text.replace("%", "").replace(",", "."))
    # même chose + remplacement
    brasseur = soup.find_all('dd', class_=['small-6 medium-9 columns'])[2].text.replace('\r\n', '').replace(' ', '').replace('\n','')
    # plus complexe on récupère dans un premier temps le contenue dans sa classe
    contenu = soup.find_all('dd', class_=['small-6 medium-9 columns js-beer-volume'])
    # puis on recupère au bon format le valeur du contenu
    contenu = int(re.findall('[0-9]+', contenu[0].text)[0])
    origine = soup.find_all('dd', class_=['small-6 medium-9 columns js-beer-country'])[0].text.replace('\r\n', '').replace(' ', '')

    beer_infos = {
        'Nom': title.text,
        'Style': style,
        'Contenu': contenu,
        'Degré d’alcool': degre,
        'Origine': origine,
        'Brasseur': brasseur
    }
    return beer_infos

print(extract_beer_infos(url))

{'Nom': 'Newcastle Brown Ale - Fût 2L The SUB', 'Style': 'Bière Brune', 'Contenu': 2, 'Degré d’alcool': 4.7, 'Origine': 'Royaume-Uni', 'Brasseur': 'ScottishNewcastle'}


In [4]:
import json

# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
BASE_URL = "https://www.beerwulf.com/"

def extract_beer_list_infos(url):
    beers = []
    
    # on récupère le contenu json de notre url
    response = requests.get(url)
    # Meme chose que précedemment on récupère le texte
    json_list_beers = json.loads(response.text)
    
    # maitenant on va récupérer les informations de chaque bière
    for item in json_list_beers.get('items'):
        # nos infos se trouvent dans content reference
        ref = item.get('contentReference')
        # on envoie dans notre donction précédente
        info = extract_beer_infos(BASE_URL + ref)
        # on l'ajoute à notre liste
        beers.append(info)
    
    return beers
beers = extract_beer_list_infos(URL_BEERLIST_FRANCE)
beers

[{'Nom': 'La Cristal IPA du Mont Blanc',
  'Style': 'IPA',
  'Contenu': 33,
  'Degré d’alcool': 4.7,
  'Origine': 'France',
  'Brasseur': 'Mont-Blanc'},
 {'Nom': 'Ninkasi Pale Ale',
  'Style': 'Bière Ambrée',
  'Contenu': 33,
  'Degré d’alcool': 4.5,
  'Origine': 'France',
  'Brasseur': 'NinkasiFabriques'},
 {'Nom': 'Desperados',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 5.9,
  'Origine': 'France',
  'Brasseur': 'Desperados'},
 {'Nom': 'Pietra',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 6.0,
  'Origine': 'France',
  'Brasseur': 'BrasseriePietra'}]

In [5]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [6]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 1.210s

OK
